## Tools y Tool Calling

#### ¿Qué son las Tools (Herramientas)?

Las herramientas son funciones en algún lenguaje de programación que encapsulan una **acción externa** al LLM. Son el puente entre la capacidad de razonamiento del LLM y el mundo real (APIs, bases de datos, sistemas de archivos).

  * **Propósito:** Proveer al LLM de **información externa** a su base de entrenamiento (como datos de la web, cálculos en tiempo real, o código).
  * **Esquema:** Las herramientas exponen su lógica y su **esquema de datos de entrada** (`a: int, b: int`) para que el LLM pueda decidir qué necesita y con qué argumentos debe llamarla.

####  Definición de Herramientas Personalizadas

Podemos definir cualquier función de Python como una herramienta usando el decorador `@tool` de LangChain. Este decorador automáticamente convierte la función, sus argumentos y su *docstring* en el esquema de datos que el LLM puede leer.

In [ ]:
import os
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

# Asegúrate de configurar tu clave API de Groq en las variables de entorno
api_key=os.environ.get("GROQ_API_KEY")

# 1. Herramienta Personalizada: Sumar dos números
@tool
def sumar_numeros(a: int, b: int) -> int:
    """Suma dos números enteros y devuelve el resultado."""
    print(f"\n⚙️ [TOOL]: Ejecutando suma de {a} + {b}")
    return a + b

# 2. Inicializar el LLM y bindear (atar) la herramienta
# Usamos un modelo optimizado y le 'enseñamos' la herramienta.
try:
    llm_with_tools = ChatGroq(model="llama-3.3-70b-versatile", api_key=api_key, temperature=0).bind_tools([sumar_numeros])
except Exception as e:
    print(f"⚠️ ERROR al inicializar ChatGroq. Asegúrate de tener la clave API configurada. Usando simulación de LLM.")
    llm_with_tools = None

pregunta_usuario = "¿Cuál es la suma de 125 y 48?"
messages = [HumanMessage(content=pregunta_usuario)]

print(f"--- FASE 1: LLM recibe pregunta y decide la acción ---")
print(f"Pregunta: {pregunta_usuario}\n")

if llm_with_tools:
    # 1. El LLM genera la Petición de Tool Call
    response_1 = llm_with_tools.invoke(messages)
else:
    # Simulación de la respuesta si falla la API
    from langchain_core.messages import ToolCall
    response_1 = AIMessage(
        content="",
        tool_calls=[ToolCall(name="sumar_numeros", args={"a": 125, "b": 48}, id="call_123")]
    )

if response_1.tool_calls:
    llamada = response_1.tool_calls[0]
    
    print("🤖 Respuesta del LLM (AIMessage):")
    print(f"  - Contenido: {response_1.content} (Vacío, porque priorizó la herramienta)")
    print(f"  - Herramienta solicitada: {llamada.get('name')}, Args: {llamada.get('args')}")

    # --- FASE 2: Ejecución de la Herramienta ---
    
    # 2. Ejecutar la función solicitada
    tool_to_call = sumar_numeros
    tool_output = sumar_numeros.invoke(llamada.get("args"))
    
    print(f"\n⚙️ Resultado de la Herramienta: {tool_output}")

    # --- FASE 3: Devolver el Resultado al LLM para la Respuesta Final ---
    
    # 3. Crear el mensaje de retroalimentación (ToolMessage)
    # Enviamos la respuesta original del LLM (response_1) + el resultado de la herramienta.
    messages.append(response_1) 
    messages.append(ToolMessage(
        content=str(tool_output),
        tool_call_id=llamada.get("id")
    ))
    
    print("\n--- FASE 4: LLM recibe el resultado y genera la respuesta final ---")
    
    # 4. Invocar el LLM de nuevo con todo el historial
    if llm_with_tools:
        final_response = llm_with_tools.invoke(messages)
    else:
        final_response = AIMessage(content=f"La suma de 125 y 48 es 173.")

    print("✅ Respuesta Final del LLM:")
    print(final_response.content)

else:
    print("El LLM respondió directamente. Esto ocurre si la pregunta es simple o si no logra mapear a una herramienta.")

### Ejemplo 1

In [ ]:
import os
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` and `b`.
    Args:
        a: First int
        b: Second int
    """
    return a / b

In [ ]:
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}

In [ ]:
from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict, Annotated
import operator


class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

In [ ]:
from langchain_core.messages import SystemMessage

llm = ChatGroq(
	model="llama-3.3-70b-versatile",
	temperature=0.0,
	api_key=os.environ.get("GROQ_API_KEY"),
)
llm = llm.bind_tools(tools)

SYSTEM_PROMPT = """
You are an kindly agent which goal is to resolve the user query using the tools that you have access to.
"""
prompt = ChatPromptTemplate.from_messages([
	("system", SYSTEM_PROMPT),
	MessagesPlaceholder(variable_name="messages")
	],
	template_format="jinja2"
)
chain = prompt | llm

def llm_call(state: MessagesState):
    return {
        "messages": [
            chain.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    } 

In [ ]:
from langchain_core.messages import ToolMessage


def tool_node(state: dict):
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}

In [ ]:
from typing import Literal
from langgraph.graph import  END


def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    messages = state["messages"]
    last_message = messages[-1]

    # Si el LLM realiza un llamado a una herramienta
    if last_message.tool_calls:
        return "tool_node"

    # Si no, responder al usuario
    return END

In [ ]:
from langgraph.graph import StateGraph, START

agent_builder = StateGraph(MessagesState)


agent_builder.add_node("llm", llm_call)
agent_builder.add_node("tool_node", tool_node)

agent_builder.add_edge(START, "llm")
agent_builder.add_conditional_edges(
    "llm",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm")

agent = agent_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Add 3 and 4.")]
messages = agent.invoke({"messages": messages})

for m in messages["messages"]:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Add 3 and 4, then multiply by 9 en divide by 3.")]
messages = agent.invoke({"messages": messages})

for m in messages["messages"]:
    m.pretty_print()